In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# URL of the page to scrape
url = "https://fbref.com/en/comps/10/2022-2023/schedule/2022-2023-Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the rows with game data
game_rows = soup.find("table", {"class": "stats_table"}).find("tbody").find_all("tr")

# Create empty lists to store data
all_data = []

# Iterate through each game
for row in game_rows:
    # Extract game ID and construct URL
    game_id = row.get("data-match")
    game_url = f"https://fbref.com/en/matches/{game_id}"

    # Send a GET request to the game URL
    game_response = requests.get(game_url)
    game_soup = BeautifulSoup(game_response.content, "html.parser")

    # Find the Shots table on the page
    shots_table = game_soup.find("div", {"id": "switcher_shots"})

    # Create empty lists to store data
    data = []
    headers = ['Minute', 'Player', 'Team', 'xG', 'PSxG', 'Result', 'Distance', 'Body Part', 'Notes', 'SCA 1 Player', 'Event', 'SCA 2 Player', 'Event']

    # Extract table rows
    rows = shots_table.find("tbody").find_all("tr")
    for row in rows:
        row_data = [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
        data.append(row_data)

    # Find the home and away teams and formations
    team_headers = game_soup.find_all("th", {"colspan": "2"})
    teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
    formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

    # Extracting the home and away teams and formations
    home_team = teams[0]
    away_team = teams[1]
    home_formation = formations[0]
    away_formation = formations[1]

    # Create a DataFrame from the shot data
    shots_df = pd.DataFrame(data, columns=headers)

    # Add the team, away team, formation, and opponent formation information to the DataFrame
    shots_df['Home_Team'] = home_team
    shots_df['Away_Team'] = away_team
    shots_df['Home Formation'] = home_formation
    shots_df['Away Formation'] = away_formation
    shots_df['is_home_shot'] = np.where(shots_df['Team'] == home_team, 1, 0)
    shots_df['is_away_shot'] = np.where(shots_df['Team'] == away_team, 1, 0)
    shots_df['Game_ID'] = game_id

    # Append the DataFrame to the list
    all_data.append(shots_df)

# Concatenate all DataFrames into one
final_df = pd.concat(all_data, ignore_index=True)

# Print the final DataFrame
print(final_df)


AttributeError: 'NoneType' object has no attribute 'find'

In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the page to scrape
url = "https://fbref.com/en/comps/10/2022-2023/schedule/2022-2023-Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the rows with game data
game_rows = soup.find("table", {"class": "stats_table"}).find("tbody").find_all("tr")

# Extract game IDs
game_ids = [row.get("id") for row in game_rows if row.get("id")]

# Print the game IDs
print(game_ids)


[]
